In [1]:
!pip install fast_bert

     |████████████████████████████████| 99 kB 534 kB/s            
     |████████████████████████████████| 97 kB 222 kB/s            
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     |████████████████████████████████| 769 kB 385 kB/s            
     |████████████████████████████████| 43 kB 285 kB/s            
  Preparing metadata (setup.py) ... done
     |████████████████████████████████| 124 kB 221 kB/s            
     |████████████████████████████████| 1.1 MB 314 kB/s            
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     |████████████████████████████████| 54 kB 439 kB/s            
  Using cached sentencepiece-0.1.96.tar.gz (508 kB)
  Preparing metadata (setup.py) ... done
  Using cached sacremoses-0.0.46-py3-none-any.whl (895 kB)
  Using cached fileloc

  ERROR: Command errored out with exit status 1:
   command: /opt/homebrew/Caskroom/miniforge/base/bin/python3.9 /opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/pip/_vendor/pep517/in_process/_in_process.py build_wheel /var/folders/ts/pmyrp_gn1tq7475xr1zcwbwm0000gn/T/tmpdkvp0mhg
       cwd: /private/var/folders/ts/pmyrp_gn1tq7475xr1zcwbwm0000gn/T/pip-install-cc85om8_/tokenizers_e9a99f7b88a54620bd616e8be4c84296
  Complete output (47 lines):
  /private/var/folders/ts/pmyrp_gn1tq7475xr1zcwbwm0000gn/T/pip-build-env-mv3en2pb/overlay/lib/python3.9/site-packages/setuptools/dist.py:493: UserWarning: Normalizing '0.8.1.rc1' to '0.8.1rc1'
    warnings.warn(tmpl.format(**locals()))
  running bdist_wheel
  running build
  running build_py
  creating build
  creating build/lib.macosx-11.0-arm64-3.9
  creating build/lib.macosx-11.0-arm64-3.9/tokenizers
  copying tokenizers/__init__.py -> build/lib.macosx-11.0-arm64-3.9/tokenizers
  creating build/lib.macosx-11.0-arm64-3.9/tokenizers

Successfully built seqeval spacy
Failed to build tokenizers sentencepiece
ERROR: Could not build wheels for tokenizers, which is required to install pyproject.toml-based projects


In [2]:
import torch
from fast_bert.data_cls import BertDataBunch
from fast_bert.learner_cls import BertLearner
from fast_bert.data_lm import BertLMDataBunch
from fast_bert.learner_lm import BertLMLearner
from fast_bert.metrics import fbeta, roc_auc
from fast_bert.prediction import BertClassificationPredictor
from pathlib import Path
import pandas as pd
import logging


logger = logging.getLogger()
device_cuda = torch.device("cuda")

ModuleNotFoundError: No module named 'fast_bert'

In [ ]:
DATA_PATH = Path('./data/')
LOG_PATH = Path('./logs/')
MODEL_PATH = Path('./model/')
LABEL_PATH = Path('./labels/')

In [ ]:
df = pd.read_csv('./data/labeled_data.csv')

In [ ]:
val_set = df.sample(frac=0.2, replace=False, random_state=42)
train_set = df.drop(index = val_set.index)
print('Nombre de commentaires dans le val_set:',len(val_set))
print('Nombre de commentaires dans le train_set:', len(train_set))
val_set.to_csv('./data/val_set.csv')
train_set.to_csv('.data/train_set.csv')

In [ ]:
labels = df.columns[2:].to_list()
with open('./labels/labels.txt', 'w') as f:
    for i in labels:
        f.write(i + "\n")

In [ ]:
df_texts = pd.read_csv('./data/raw_data.csv')
all_texts = df_texts['caption'].to_list()
print('Nombre de commentaires:', len(all_texts))

### Création de LMDataBunch

In [ ]:
databunch_lm = BertLMDataBunch.from_raw_corpus(
                    data_dir=DATA_PATH,
                    text_list=all_texts,
                    tokenizer='camembert-base',
                    batch_size_per_gpu=16,
                    max_seq_length=512,
                    multi_gpu=False,
                    model_type='camembert-base',
                    logger=logger)

### Création de LMLearner

In [ ]:
lm_learner = BertLMLearner.from_pretrained_model(
                            dataBunch=databunch_lm,
                            pretrained_path='camembert-base',
                            output_dir=MODEL_PATH,
                            metrics=[],
                            device=device_cuda,
                            logger=logger,
                            multi_gpu=False,
                            logging_steps=50,
                            fp16_opt_level="O2")

In [ ]:
lm_learner.fit(epochs=30,
            lr=1e-4,
            validate=True,
            schedule_type="warmup_cosine",
            optimizer_type="adamw")

In [ ]:
lm_learner.validate()

In [ ]:
learner.save_model()

### Création de databunch pour la classification

In [ ]:
databunch = BertDataBunch(DATA_PATH, LABEL_PATH,
                          tokenizer='camembert-base',
                          train_file='train_set.csv',
                          val_file='val_set.csv',
                          label_file='labels.txt',
                          text_col='review',
                          label_col=['cadre/atmosphère','probleme technique',"temps d'attente",'accueil/relation commerciale'],
                          batch_size_per_gpu=16,
                          max_seq_length=512,
                          multi_gpu=False,
                          multi_label=True,
                          model_type='camembert-base')

### Création de Learner

In [ ]:
metrics = [{'name': 'fbeta', 'function': fbeta}, {'name': 'roc_auc', 'function': roc_auc}]
OUTPUT_DIR = Path('./finetuned_model')
WGTS_PATH = Path('model/model_out/pytorch_model.bin')

In [ ]:
cl_learner = BertLearner.from_pretrained_model(
                        databunch,
                        pretrained_path='model/model_out',
                        metrics=metrics,
                        device=device_cuda,
                        logger=logger,
                        output_dir=OUTPUT_DIR,
                        finetuned_wgts_path=WGTS_PATH,
                        warmup_steps=300,
                        multi_gpu=False,
                        multi_label=True,
                        is_fp16=True,
                        logging_steps=50)

In [ ]:
cl_learner.fit(epochs=30,
            lr=9e-5,
            validate=True,
            schedule_type="warmup_cosine",
            optimizer_type="adamw")

In [ ]:
cl_learner.validate()

In [ ]:
class_learner.save_model()

### Prédictions

In [ ]:
predictor = BertClassificationPredictor(
                model_path='finetuned_model/model_out',
                label_path='labels/',
                multi_label=True,
                model_type='camembert-base',
                do_lower_case=False)

In [ ]:
predictor.predict("Texte à classer")